<a href="https://colab.research.google.com/github/DeepanJayaraman/Transfer_learning/blob/main/Image_classification_using_transfer_learning_and_tensor_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install tensor flow hub

In [46]:
!pip install "tensorflow_hub"

# Load libraries

In [47]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

# Import mobilenet model for transfer learning

In [48]:
Class_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/2"
Im_res = 224

Set predefined layer for TL

In [49]:
TL_layer = hub.KerasLayer(Class_URL,input_shape=(Im_res,Im_res,3))


Load data set for classification from tf data sets
cats_dogs

In [50]:
(train_img,vald_img), info = tfds.load('cats_vs_dogs',as_supervised=True,with_info=True,split=['train[:80%]', 'train[80%:]'])

Data size and no. of features

In [51]:
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Randomly check some image sizes

In [52]:
for i,image in enumerate(train_img.take(5)):
  print("image: {}, Size: {}".format(i+1,image[0].shape))

image: 1, Size: (262, 350, 3)
image: 2, Size: (409, 336, 3)
image: 3, Size: (493, 500, 3)
image: 4, Size: (375, 500, 3)
image: 5, Size: (240, 320, 3)


Write funciton to resize the image

In [53]:
def im_resize(img,label):
  img = tf.image.resize(img,(Im_res,Im_res))/255 # Normalize
  return img,label

In [54]:
BATCH_SIZE = 32

train_batches      = train_img.shuffle(num_examples//4).map(im_resize).batch(BATCH_SIZE).prefetch(1)
validation_batches = vald_img.map(im_resize).batch(BATCH_SIZE).prefetch(1)

In [55]:
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

In [56]:
# Batch size extract from the data set
feature_batch = TL_layer(image_batch)

# Freeze the layer: So that the tranfer learning model parameters will not change

In [57]:
TL_layer.trainable=False

Layer assigning for output

In [58]:
out_layer = tf.keras.layers.Dense(2)

Attach the layers

In [59]:
model = tf.keras.Sequential([TL_layer,out_layer])

# Training model

Compile the model

In [60]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Fit the model

In [ ]:
History = model.fit(train_batches,
          epochs=8,
          validation_data=validation_batches)

Epoch 1/8
582/582 [==============================] - 1188s 2s/step - loss: 0.1331 - accuracy: 0.9529 - val_loss: 0.0357 - val_accuracy: 0.9890
Epoch 2/8
582/582 [==============================] - ETA: 0s - loss: 0.0352 - accuracy: 0.9892

Check accuracy and losses

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()